In [5]:
from faker import Faker
import json
import sqlite3
import random
import secrets

In [6]:
# fetch the products id, names, price from the database
def fetch_products():
    conn = sqlite3.connect('database/database.db')
    c = conn.cursor()
    c.execute('SELECT * FROM products')
    products = c.fetchall()
    conn.close()
    return products

# randomly select a product id
def random_product_id(products):
    return random.choice(products)[0]

# get the product name from the product id
def get_product_name(products, product_id):
    for product in products:
        if product[0] == product_id:
            return product[1]

# get the product price from the product id
def get_product_price(products, product_id):
    for product in products:
        if product[0] == product_id:
            return product[2]

# order id 36 hexes
def order_crypto_id():
    return secrets.token_hex(36)

# order quantity
def order_quantity():
    return random.randint(1, 15)

# order sku
def order_sku():
    sku = [
        'CTN', # cartons
        'OUT', # outer
        'BTL', # bottle
        'PKT', # packet
        'BDL' # bundle
    ]
    
    weight = [
        0.1,
        0.25,
        0.2,
        0.35,
        0.1
    ]
    
    return random.choices(sku, weight,k=1)[0]

# order status
def order_status():
    status = [
        'pending',
        'processing',
        'completed',
        'cancelled'
    ]
    
    weight = [
        0.3,
        0.2,
        0.4,
        0.1
    ]
    
    return random.choices(status, weight, k=1)[0]

# order date
def order_date():
    fake = Faker()
    date_obj = fake.date_this_year()
    return date_obj.strftime('%Y-%m-%d')

In [7]:
def create_order():
    products = fetch_products()
    product_id = random_product_id(products)
    amount = order_quantity()
    
    order = {
        'id': order_crypto_id(),
        'product_id': product_id,
        'name': get_product_name(products, product_id),
        'price': get_product_price(products, product_id),
        'total_price': get_product_price(products, product_id) * amount,
        'quantity': amount,
        'sku': order_sku(),
        'status': order_status(),
        'date': order_date()
    }
    return order

def generate_multiple_orders(n):
    orders = [create_order() for _ in range(n)]
    return json.dumps(orders, indent=4)

In [8]:
# insert the orders into the database
conn = sqlite3.connect('database/database.db')
c = conn.cursor()

c.execute(
    '''
    CREATE TABLE IF NOT EXISTS orders (
        id TEXT PRIMARY KEY,
        product_id TEXT,
        name TEXT,
        price REAL,
        total_price REAL,
        quantity INTEGER,
        sku TEXT,
        status TEXT,
        date DATE,
        FOREIGN KEY (product_id) REFERENCES products(id)
    )
    '''
)

json_data = generate_multiple_orders(300)

# save the json data as json
with open('json_data/orders.json', 'w') as f:
    f.write(json_data)
    
for order in json.loads(json_data):
    c.execute(
        '''
        INSERT INTO orders (id, product_id, name, price, total_price, quantity, sku, status, date)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        ''',
        (
            order['id'],
            order['product_id'],
            order['name'],
            order['price'],
            order['total_price'],
            order['quantity'],
            order['sku'],
            order['status'],
            order['date']
        )
    )
    
conn.commit()
conn.close()